In [ ]:
### Calculate MHT and decomposition

import numpy as np
import netCDF4 as nc
import glob
from scipy import nanmean
from scipy import interpolate
import scipy.ndimage
import matplotlib.pyplot as plt

plt.style.use('classic')
plt.figure(figsize=(15,10))

grid_file = nc.Dataset('/Model output-OG_2100/grid.nc')
Y = grid_file.variables['Y'][:]
Z = grid_file.variables['Z'][:]
Yp1 = grid_file.variables['Yp1'][:]
dxG = grid_file.variables['dxG'][:]
HFacS=grid_file.variables['HFacS'][:]
vmask=HFacS
vmask[vmask>0]=1

# total heat transport
VVELTH_file=nc.Dataset('/Model output-OG_2100/VVELTH.0085224960.nc')
VVELTH=VVELTH_file.variables['VVELTH'][0,:,:,:]
VVELTH_zonal_sum=np.sum(VVELTH*dxG,axis=2)*1000*4000## zonal integrated total MHT

# eddy heat transport
VVEL_file=nc.Dataset('/Model output-OG_2100/VVEL.0085224960.nc')
VVEL=VVEL_file.variables['VVEL'][0,:,:,:]
THETA_file=nc.Dataset('/Model output-OG_2100/THETA.0085224960.nc')
THETA=THETA_file.variables['THETA'][:]

THETA_mean=np.zeros((42,937,126))
THETA_mean[:,1:936,:] = 0.5 * (THETA[0,:,:-1,:] + THETA[0,:,1:,:]) ## mean temperature on v point
THETA_mean[:,0,:]=0.5*(THETA[0,:,0,:]+THETA[0,:,1,:])
THETA_mean[:,-1,:]=0.5*(THETA[0,:,935,:]+THETA[0,:,-1,:])

MHT_mean=VVEL*THETA_mean ### mean V * mean T -> mean part
MHT_mean_zonal_sum=np.sum(MHT_mean*dxG,axis=2)*1000*4000## <mean V * mean T>

MHT_eddy=VVELTH-MHT_mean # eddy MHT
MHT_eddy_zonal_sum=np.sum(MHT_eddy*dxG,axis=2)*1000*4000

# <V>*<T> zonal mean flow
VVEL_zonal_mean=np.sum(VVEL*vmask,axis=2)/np.sum(vmask,axis=2)
THETA_mean_zonal_mean=np.sum(THETA_mean*vmask,axis=2)/np.sum(vmask,axis=2)
MHT_zonal_mean_flow=VVEL_zonal_mean*THETA_mean_zonal_mean
MHT_zonal_mean_flow=np.expand_dims(MHT_zonal_mean_flow,2)
MHT_zonal_mean_flow=np.repeat(MHT_zonal_mean_flow, 126, 2)
MHT_zonal_mean_flow_sum=np.sum(MHT_zonal_mean_flow*dxG,axis=2)*1000*4000

# standing meanders heat transport (<mean V * mean T>-<V>*<T>)
MHT_standing_zonal_sum = MHT_mean_zonal_sum - MHT_zonal_mean_flow_sum


## Plot MHT

plt.pcolormesh(Yp1,Z,VVELTH_zonal_sum/1e10,cmap='seismic',vmin=-1,vmax=1)
cbar=plt.colorbar(pad=0.03, extend='both')
cbar.set_label('Heat transport [${^\circ}$C m s${^{-1}}$]', rotation=270, labelpad=30, fontsize=20)
cbar.ax.tick_params(labelsize=18)
plt.xlabel('Latitude [${^\circ}$N]', fontsize=20)
plt.ylabel('Depth [m]', fontsize=32)
plt.xticks(fontsize=28)
plt.yticks(fontsize=28)
plt.xlim([-60,60])
plt.ylim([-4875,0])

## Calculate zonally and vertically integrated MHT and decompositions

VVELTH_zonal_inte_z_2100OG=np.sum(VVELTH_zonal_sum*drF,axis=0)*1000*4000 
MHT_eddy_zonal_z_2100OG=np.sum(MHT_eddy_zonal_sum*drF,axis=0)*1000*4000 
MHT_zonal_mean_flow_z_2100OG=np.sum(MHT_zonal_mean_flow_sum*drF,axis=0)*1000*4000
MHT_standing_zonal_z_2100OG=np.sum(MHT_standing_zonal_sum*drF,axis=0)*1000*4000

plt.subplot(5,2,7)
plt.plot(Yp1,VVELTH_zonal_inte_z_2100OG/1e12,linewidth=2,color="black",label='Net')
plt.plot(Yp1,MHT_zonal_mean_flow_z_2100OG/1e12,linewidth=2,color=colors[3],label='Mean flow')
plt.plot(Yp1,MHT_standing_zonal_z_2100OG/1e12,linewidth=2,color=colors[9],label='Standing meanders')
plt.plot(Yp1,MHT_eddy_zonal_z_2100OG/1e12,linewidth=2,color=colors[2],label='Transient eddies')

plt.ylabel('MHT$_{Z}$ [TW]', fontsize=26)
plt.xlabel('Latitude [${^\circ}$N]', fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.yticks([-20,-10,0,10,20])
plt.xlim([-60,60])
plt.ylim([-30,30])


In [ ]:
###  Calculate Channel integrated VHT

import numpy as np
import netCDF4 as nc
import glob
from scipy import nanmean
from scipy import interpolate
import scipy.ndimage
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from collections import OrderedDict

import seaborn as sns
plt.figure(figsize=(15,10))
colors=sns.color_palette("colorblind")

### 2100DP case ###
grid_file = nc.Dataset('/Model output-OG_2100/grid.nc')
Y = grid_file.variables['Y'][:]
Z = grid_file.variables['Z'][:]
dxG = grid_file.variables['dxG'][:]
dyG = grid_file.variables['dyG'][:]
Yp1 = grid_file.variables['Yp1'][:]
rA=grid_file.variables['rA'][:]
HFacC=grid_file.variables['HFacC'][:]
tmask=HFacC
tmask[tmask>0]=1

# total vertical heat flux",
ADVr_TH_file=nc.Dataset('/Model output-OG_2100/ADVr_TH.0087868800.nc')
ADVr_TH=ADVr_TH_file.variables['ADVr_TH'][0,:,:,:]
ADVr_TH_zonal=np.sum(ADVr_TH,axis=2)## sum(WTrA)
ADVr_TH_zonal_meri=np.sum(ADVr_TH_zonal[:,0:191],axis=1)## sumsum(WTrA)
ADVr_TH_zonal_meri=ADVr_TH_zonal_meri*1000*4000 ## need to time by rho0, Cp to get watt

# eddy vertical heat flux
WVEL_file=nc.Dataset('/Model output-OG_2100/WVEL.0087868800.nc')
WVEL=WVEL_file.variables['WVEL'][0,:,:,:]

THETA_file=nc.Dataset('/Model output-OG_2100/THETA.0087868800.nc')
THETA=THETA_file.variables['THETA'][0,:,:,:]

THETA_vertical_mean = 0.5 * (THETA[:-1,:,:] + THETA[1:,:,:])
THETA_vertical_mean_s=THETA_vertical_mean[0,:,:]
THETA_vertical_mean_s=np.reshape(THETA_vertical_mean_s,[1,936,126])
THETA = np.concatenate((THETA_vertical_mean_s,THETA_vertical_mean),axis=0)## add surface face

VHT_mean=WVEL*THETA*rA ### mean W * mean T * rA
VHT_mean_zonal=np.sum(VHT_mean,axis=2) ### <W*T*rA>

VHT_mean_zonal_meri=np.sum(VHT_mean_zonal,axis=1)### <<W*T*rA>>
VHT_mean_zonal_meri=VHT_mean_zonal_meri*1000*4000 
VHT_eddy=ADVr_TH-VHT_mean # eddy vertical heat flux
VHT_eddy_zonal=np.sum(VHT_eddy,axis=2)
VHT_eddy_zonal_meri=np.sum(VHT_eddy_zonal[:,0:191],axis=1)
VHT_eddy_zonal_meri=VHT_eddy_zonal_meri*1000*4000 

# zonal mean vertical heat flux

WVEL_zonal= np.sum(WVEL*tmask,axis=2)/np.sum(tmask,axis=2)
THETA_zonal= np.sum(THETA*tmask,axis=2)/np.sum(tmask,axis=2)
WVEL_zonal=np.expand_dims(WVEL_zonal,2).repeat(126,axis=2)
THETA_zonal=np.expand_dims(THETA_zonal,2).repeat(126,axis=2)

THETA_zonal_vertical_mean = 0.5 * (THETA_zonal[:-1,:,:] + THETA_zonal[1:,:,:]) 
THETA_zonal_vertical_mean_s=THETA_zonal_vertical_mean[0,:,:]
THETA_zonal_vertical_mean_s=np.reshape(THETA_zonal_vertical_mean_s,[1,936,126])
THETA_zonal = np.concatenate((THETA_zonal_vertical_mean_s,THETA_zonal_vertical_mean),axis=0)

VHT_zonal_mean=WVEL_zonal * THETA_zonal*rA ### <W>*<T>*rA
VHT_zonal_mean_zonal=np.sum(VHT_zonal_mean,axis=2)
VHT_zonal_mean_zonal_meri=np.sum(VHT_zonal_mean_zonal[:,0:191],axis=1)
VHT_zonal_mean_zonal_meri=VHT_zonal_mean_zonal_meri*1000*4000

# standing heat transport (=<mean VT>-zoanl mean)
VHT_standing_zonal = VHT_mean_zonal - VHT_zonal_mean_zonal
VHT_standing_zonal_meri=np.sum(VHT_standing_zonal[:,0:191],axis=1) 
VHT_standing_zonal_meri=VHT_standing_zonal_meri*1000*4000

control=np.zeros(42)
plt.plot(control,Z,linewidth=1,color="grey",linestyle=":")
plt.plot(ADVr_TH_zonal_meri/1e12,Z,linewidth=2,color="black",label='Net')
plt.plot(VHT_zonal_mean_zonal_meri/1e12,Z,linewidth=2,color=colors[3],label='Mean flow')
plt.plot(VHT_standing_zonal_meri/1e12,Z,linewidth=2,color=colors[9],label='Standing meanders')
plt.plot(VHT_eddy_zonal_meri/1e12,Z,linewidth=2,color=colors[2],label='Transient eddies')
plt.ylim([-5000,0])
plt.xlim([-6,6])
plt.ylabel('Depth [m]', fontsize=26)
plt.yticks([0,-1000,-2000,-3000,-4000])
plt.xticks(fontsize=20)
plt.xticks([-4,-2,0,2,4])
plt.yticks(fontsize=20)
plt.ylabel('Depth [m]', fontsize=26)
plt.yticks([0,-1000,-2000,-3000,-4000])
plt.xlabel('VHT [TW]', fontsize=26)
plt.legend(ncol=1, loc='lower right',fontsize=12)

